In [1]:
import csv

In [2]:
race_name = 'Squad_Test'

n_riders = 6
n_subs = 2
gender = 'W'
max_cost = 54

In [3]:
def initialise (n_teams, default):
    set = []
    for d in range(n_teams):
        set.append(default)
    return set

In [4]:
def checker (rs, s, n_loops):
    unique = True
    for sub in range(n_loops):
        for rider in range(n_riders):
            if s[sub] == rs[rider]:
                unique = False
                break
    
    return unique

In [5]:
def new_rs (rs, s, index):
    rs_sub = []
    for r in rs:
        rs_sub.append(r)
    for sub in range(len(s)):
        rs_sub[index[sub]] = s[sub]
    rs_new = sorted(rs_sub)
    return rs_new

In [6]:
def adding (info, rs, col):
    sub = 0
    for r in rs:
        sub += info[r][col]
        
    return sub

In [7]:
def cost_checker (comb, cost):
    if comb > cost:
        check = False
    else:
        check = True
        
    return check

In [8]:
def loop_setup (n_loops):
    s = []
    for n in range(n_loops):
        s.append(n)
    
    return s

In [9]:
def s_loop (s, s_complete, n_loops, n_races, n_all):
    counter = 0
    
    for n in range(n_races):
        if s_complete[n_races-n-1] == True:
            counter += 1
        else:
            index = n_races-n-1
            break
            
    if counter == n_races:
        s_finished = True
    else:
        [s[index], s_complete[index]] = s_iterate(s[index], n_loops[index], n_all)
        for n in range(index+1,n_races):
            s[n] = loop_setup(n_loops[n])
            s_complete[n] = False
        s_finished = False
    
    return s, s_complete, s_finished

In [10]:
def loops_loop (n_loops, loops_complete, n_races, n_riders, n_subs):

    if loops_complete[n_races-1] == False:
        n_loops[n_races-1] += 1
        if sum(n_loops) == n_subs or n_loops[n_races-1] == n_riders:
            loops_complete[n_races-1] = True
        loops_finished = False
    else:
        counter = 0
        for n in range(n_races):
            if loops_complete[n_races-n-1] == False:
                index = n_races-n-1
                break
            else:
                counter += 1
    
        if counter != n_races:
            loops_finished = False
            n_loops[index] += 1
            for n in range(index+1,n_races):
                n_loops[n] = 0

            if n_loops[index] == n_riders:
                loops_complete[index] = True
            
            if sum(n_loops[0:index+1]) == n_subs:
                loops_complete[index] = True
                for n in range(index+1,n_races):
                    loops_complete[n] = True
            else:
                for n in range(index+1,n_races):
                    loops_complete[n] = False
        else:
            loops_finished = True
    
    return n_loops, loops_complete, loops_finished

In [11]:
def s_iterate (s, n_loops, n_all):
    counter = 0
    
    for n in range(n_loops):
        if s[n_loops-n-1] == n_all-n-1:
            counter += 1
        else:
            r = n_loops-n-1
            break
            
    if counter == n_loops:
        s_complete = True
    else:
        s[r] = s[r] + 1
        for n in range(r+1,n_loops):
            s[n] = s[n-1] + 1
        s_complete = False
    
    return s, s_complete

In [12]:
def riders_iterate (riders, n_loops, n_riders):
    counter = 0
    
    for n in range(n_loops):
        if riders[n_loops-n-1] == n_riders-n-1:
            counter += 1
        else:
            r = n_loops-n-1
            break
            
    if counter == n_loops:
        riders_complete = True
    else:
        riders[r] = riders[r] + 1
        for n in range(r+1,n_loops):
            riders[n] = riders[n-1] + 1
        riders_complete = False
    
    return riders, riders_complete

In [20]:
def main_loop (info, race, rs, s, s_complete, s_incomplete, n_loops, riders, riders_complete, riders_incomplete, n_riders, n_all):
    unique = checker(rs[race-1], s, n_loops)
    while unique == False and s_complete == False:
        [s, s_complete] = s_iterate(s, n_loops, n_all)
        unique = checker(rs[race-1], s, n_loops)
    if s_complete == False:
        rs[race] = new_rs(rs[race-1], s, riders)
        comb_costs = adding(info, rs[race], 1)
        cost_check = cost_checker(comb_costs, max_cost)
        while cost_check == False and riders_complete == False:
            [riders, riders_complete] = riders_iterate(riders, n_loops, n_riders)
            if riders_complete == False:
                rs[race] = new_rs(rs[race-1], s, riders)
                comb_costs = adding(info, rs[race], 1)
                cost_check = cost_checker(comb_costs, max_cost)
    
    return rs, s, s_complete, s_incomplete, riders, riders_complete, riders_incomplete, comb_costs

In [14]:
def splitting (inter, races, n_all):
    info = initialise(n_races, [])
    l = 0
    for race in races:
        split = []
        for row in range(n_all):
            split.append([inter[row]['Names'], inter[row]['Cost'], inter[row][race]])
        info[l] = split
        l += 1
        
    return info

In [15]:
def new_team (info, rs):
    sub = []
    for r in rs:
        sub.append(info[r][0])
        
    return sub

In [16]:
if gender == 'W':
    races = ['Race1', 'Race2']
if gender == 'M':
    races = ['Race1', 'Race2', 'Race3']
n_races = len(races)

In [17]:
in_file = open(str(race_name) + '_' + str(gender) + '.txt', 'r')
inter = []
l = 0

data = csv.DictReader(in_file)
for row in data:
    inter.append(row)
    inter[l]['Cost'] = int(inter[l]['Cost'])
    for race in races:
        inter[l][race] = int(inter[l][race])
    inter[l]['Total'] = int(inter[l]['Total'])
    l += 1
in_file.close()

n_all = len(inter)

info = splitting(inter, races, n_all)

In [18]:
def riders_loop (riders, riders_complete, n_loops, n_races, n_riders):
    counter = 0
    
    for n in range(n_races):
        if riders_complete[n_races-n-1] == True:
            counter += 1
        else:
            index = n_races-n-1
            break
            
    if counter == n_races:
        riders_finished = True
    else:
        [riders[index], riders_complete[index]] = riders_iterate(riders[index], n_loops[index], n_riders)
        for n in range(index+1,n_races):
            riders[n] = loop_setup(n_loops[n])
            riders_complete[n] = False
        riders_finished = False
    
    return riders, riders_complete, riders_finished

In [29]:
rs = [[0,1,2], [], []]
s = [[], [0], [0]]
s_complete = [True, False, False]
riders = [[], [0], [0]]
riders_complete = [True, False, False]
comb_costs = [0, 0, 0]
n_loops = [0, 1, 1]
n_riders = 2
comb_scores = [0, 0, 0]
squad_teams = [[], [], []]
squad_scores = [0, 0, 0]
squad_costs = [0, 0, 0]
squad_total = 0
n_races = 2

In [30]:
s_incomplete = False
riders_incomplete = False
for race in range(1,n_races):
    [rs, s[race], s_complete[race], s_incomplete, riders[race], riders_complete[race], riders_incomplete, comb_costs[race]] = main_loop(info[race], race, rs, s[race], s_complete[race], s_incomplete, n_loops[race], riders[race], riders_complete[race], riders_incomplete, n_riders, n_all)
    comb_scores[race] = adding(info[race], rs[race], 2)
comb_total = sum(comb_scores)
if comb_total > squad_total and s_incomplete == False and riders_incomplete == False:
    for race in range(n_races):
        squad_teams[race] = new_team(info[race], rs[race])
        squad_scores[race] = comb_scores[race]
        squad_costs[race] = comb_costs[race]
        squad_total = comb_total
[riders, riders_complete, riders_finished] = riders_loop(riders, riders_complete, n_loops, n_races, n_riders)

print(s)
print(s_complete, s_incomplete)
print(riders)
print(riders_complete, riders_incomplete)

[[], [2], [0]]
[True, False, False] False
[[], [1], [0]]
[True, False, False] False


In [41]:
riders_complete = [True]
riders_finished = False
riders = [[]]
n_loops = [0]
n_riders = 3
n_races = 1

while riders_finished == False:
    [riders, riders_complete, riders_finished] = riders_loop(riders, riders_complete, n_loops, n_races, n_riders)
print(riders)
print(riders_complete)
print(riders_finished)

[[]]
[True]
True


In [ ]:
while riders_finished == False:
    [riders, riders_complete, riders_finished] = riders_loop(riders, riders_complete, n_loops, n_races, n_riders)